In [1]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (0 entries)


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [3]:
#importing dataframe
path = r'C:\Users\lenovo\09-2024 CareerFoundry'
cust = pd.read_csv(os.path.join(path, '02 Data' , 'customers.csv'))

In [4]:
cust.head

<bound method NDFrame.head of         user_id First Name    Surnam  Gender           STATE  Age date_joined  \
0         26711    Deborah  Esquivel  Female        Missouri   48    1/1/2017   
1         33890   Patricia      Hart  Female      New Mexico   36    1/1/2017   
2         65803    Kenneth    Farley    Male           Idaho   35    1/1/2017   
3        125935   Michelle     Hicks  Female            Iowa   40    1/1/2017   
4        130797        Ann   Gilmore  Female        Maryland   26    1/1/2017   
...         ...        ...       ...     ...             ...  ...         ...   
206204   168073       Lisa      Case  Female  North Carolina   44    4/1/2020   
206205    49635     Jeremy   Robbins    Male          Hawaii   62    4/1/2020   
206206   135902      Doris  Richmond  Female        Missouri   66    4/1/2020   
206207    81095       Rose   Rollins  Female      California   27    4/1/2020   
206208    80148    Cynthia     Noble  Female        New York   55    4/1/2020  

In [5]:
#renaming the columns
cust.rename(columns={'First Name': 'first_name', 'Surnam': 'last_name', 'n_dependants':'number_of_dependants','fam_status':'marital_status'}, inplace=True)

In [6]:
#finding missing values
cust.isnull().sum()

user_id                     0
first_name              11259
last_name                   0
Gender                      0
STATE                       0
Age                         0
date_joined                 0
number_of_dependants        0
marital_status              0
income                      0
dtype: int64

There are 11259 customer records with the first name missing. These rows will be removed from further analysis.

In [7]:
#filtering records where the first name is not NULL
cust = cust[cust['first_name'].isnull() == False]

In [8]:
#checking for duplicates
cust_dups = cust[cust.duplicated()]
cust_dups

,user_id,first_name,last_name,Gender,STATE,Age,date_joined,number_of_dependants,marital_status,income


There are no duplicates in the customer dataframe!

Consistency checks for customer

In [9]:
for col in cust.columns.tolist():
  weird = (cust[[col]].applymap(type) != cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (cust[weird]) > 0:
    print (col)

Cross-checking the data types of the columns

In [10]:
cust.dtypes

user_id                  int64
first_name              object
last_name               object
Gender                  object
STATE                   object
Age                      int64
date_joined             object
number_of_dependants     int64
marital_status          object
income                   int64
dtype: object

In [11]:
cust['Age'] = cust['Age'].astype('int8')
cust['number_of_dependants'] = cust['number_of_dependants'].astype('int8')

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
cust['marital_status'].value_counts()

married                             137012
single                               32070
divorced/widowed                     16682
living with parents and siblings      9186
Name: marital_status, dtype: int64

In [14]:
cust['Gender'].value_counts()

Male      98631
Female    96319
Name: Gender, dtype: int64

IMPORTING THE INSTACART DATAFRAME

In [16]:
# Import the rest of prepared Instacart data
ords_prods =pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_prods_frequency.pkl'))

In [17]:
ords_prods.dtypes
#changing the datatypes to save memory during merging
ords_prods['order_number'] = ords_prods['order_number'].astype('int8')
ords_prods['product_id'] = ords_prods['product_id'].astype('int8')
ords_prods['aisle_id'] = ords_prods['aisle_id'].astype('int8')
ords_prods['department_id'] = ords_prods['department_id'].astype('int8')
ords_prods['department_id'] = ords_prods['department_id'].astype('int8')
ords_prods['prices'] = ords_prods['prices'].astype('float32')
ords_prods['max_order'] = ords_prods['max_order'].astype('float32')

Breaking Customers.csv into chunks and merging it with the Instacart dataframe

In [19]:
%%time

# Creating an empty shell to save the result
df_result = pd.DataFrame(columns=(ords_prods.columns.append(cust.columns)).unique())
df_result.to_csv(os.path.join(path, '02 Data', 'Prepared Data', "ords.csv"),index_label=False)

# Deleting customer dataframe to save memory
del(cust)

# The basic idea is that we'll now load the large data set into chunks and then create a function that will 
# iteratively merge those chunks to the main data set 


# Define a function that will merge the chunks
def preprocess(x):
    ords = pd.merge(ords_prods,x, left_on = "user_id", right_on = "user_id")
    ords.to_csv(os.path.join(path, '02 Data', 'Prepared Data', "ords.csv"),mode="a",header=False,index=False)

    
reader = pd.read_csv(os.path.join(path, '02 Data', 'customers.csv'), chunksize=10000) # chunksize depends on you colsize

[preprocess(r) for r in reader]

Wall time: 8min 22s


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

Q. The merged dataframe (Instacart + Customers)

In [3]:
cust.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,first_name,last_name,Gender,STATE,Age,date_joined,number_of_dependants,marital_status,income
0,2293453,19,2,5,14,6,-60,3,0,Soda,...,Frequent customer,Jane,Salazar,Female,Louisiana,62,2/3/2017,1,married,151988
1,1973799,19,5,6,12,8,-60,15,1,Soda,...,Frequent customer,Jane,Salazar,Female,Louisiana,62,2/3/2017,1,married,151988
2,532817,19,7,4,17,6,-60,1,1,Soda,...,Frequent customer,Jane,Salazar,Female,Louisiana,62,2/3/2017,1,married,151988
3,2474845,19,6,5,13,20,72,5,0,Oats & Honey Gluten Free Granola,...,Frequent customer,Jane,Salazar,Female,Louisiana,62,2/3/2017,1,married,151988
4,86918,19,9,5,16,8,-85,16,0,Granny Smith Apples,...,Frequent customer,Jane,Salazar,Female,Louisiana,62,2/3/2017,1,married,151988
